In [1]:
from pprint import pprint
import time
from datetime import datetime
from dateutil import tz
import pytz
import json
from uuid import UUID
import asyncio
from copy import deepcopy
from utils.wynncraft_response_utils import WynncraftResponseUtils as WynnUtils
from request.request_wynncraft import RequestWynncraft
from settings import VindicatorTables
from mojang import API
from database.vindicator_database import VindicatorDatabase

from typing import TYPE_CHECKING, Any, Dict, List

if TYPE_CHECKING:
    from aiohttp import ClientResponse

In [2]:
async def ainit_all():
    await RequestWynncraft.ainit()
    await VindicatorDatabase.ainit()

asyncio.create_task(ainit_all())

<Task pending name='Task-5' coro=<ainit_all() running at C:\Users\user\AppData\Local\Temp\ipykernel_1952\1110673793.py:1>>

In [3]:
def test1():
    query1 = "INSERT INTO test (timestamp) VALUES (%(timestamp)s)"
    params1 = [{"timestamp": datetime.fromtimestamp(ts)} for ts in range(int(time.time()), int(time.time()) + 1_000_000)]
    coro1 = VindicatorDatabase.write_many(query1, params1)
    task1 = asyncio.create_task(coro1)
    return task1

def test2():
    query2 = "INSERT INTO test_int (timestamp) VALUES (%(timestamp)s)"
    params2 = [{"timestamp": ts} for ts in range(int(time.time()), int(time.time()) + 1_000_000)]
    coro2 = VindicatorDatabase.write_many(query2, params2)
    task2 = asyncio.create_task(coro2)
    return task2

def test3():
    query = "SELECT timestamp FROM test LIMIT 10"
    coro1 = VindicatorDatabase.read_all(query)
    task1 = asyncio.create_task(coro1)
    return task1

def test4():
    query = "SELECT timestamp FROM test_int LIMIT 10"
    coro1 = VindicatorDatabase.read_all(query)
    task1 = asyncio.create_task(coro1)
    return task1

def test5():
    query = (
        "INSERT INTO player_main_info (uuid, latest_username, timestamp) "
        "VALUES (%(uuid)s, %(latest_username)s, %(timestamp)s)"
    )
    with open("uuid-username-timestamp.json", "r") as f:
        data = json.load(f)

    for record in data:
        record["uuid"] = UUID(record["uuid"]).bytes

    params = data
    coro = VindicatorDatabase.write_many(query, params)
    task = asyncio.create_task(coro)
    return task

def test6(name: str):
    coro = RequestWynncraft.get_player_stats(name)
    task = asyncio.create_task(coro)
    return task

async def test7(username: str):
    params = {'username': username}
    query = f"SELECT uuid FROM {VindicatorTables.PLAYER_MAIN_INFO} WHERE latest_username = %(username)s"
    uuid = await VindicatorDatabase.read_all(query=query, params=params)
    return uuid

In [4]:
task = test6("parrottv1")

In [5]:
print(f"{task.done()=}")
pprint(task.result())

task.done()=True


Exception: Error 500 - Internal Server Error

In [21]:
asyncio.create_task(VindicatorDatabase._close_connection())
asyncio.create_task(RequestWynncraft._session._close())

<Task pending name='Task-11' coro=<RequestBase._close() running at c:\Users\user\VSCodeProjects\Vindicator\src\request\request_base.py:34>>

In [28]:
def parse_gamemode(gamemodebyte: bytes) -> List[str]:
    ret = []

    byte_value = int.from_bytes(gamemodebyte, byteorder='big')  # Byte to int
    is_hardcore = (byte_value & (1 << 0)) != 0
    is_ultimate = (byte_value & (1 << 1)) != 0
    is_ironman = (byte_value & (1 << 2)) != 0
    is_craftsman = (byte_value & (1 << 3)) != 0
    is_hunted = (byte_value & (1 << 4)) != 0

    if is_hardcore:
        ret.append("hardcore")
    if is_ultimate:
        ret.append("ultimate")
    if is_ironman:
        ret.append("ironman")
    if is_craftsman:
        ret.append("craftsman")
    if is_hunted:
        ret.append("hunted")

    return ret

def format_gamemodes(gamemodeslist: List[str]) -> bytes:
    gamemode_byte = 0  # Initialize a byte to 0

    gamemode_mapping = {
        "hardcore": 0,
        "ultimate": 1,
        "ironman": 2,
        "craftsman": 3,
        "hunted": 4
    }

    # Set the corresponding bits in the byte
    for gamemode in gamemodeslist:
        if gamemode not in gamemode_mapping:
            raise ValueError(f"Invalid gamemode: {gamemode}")

        bit_position = gamemode_mapping[gamemode]
        gamemode_byte |= (1 << bit_position)

    ret = gamemode_byte.to_bytes(1, byteorder='big')
    return ret

one = parse_gamemode(bytes([0b10000]))
two = format_gamemodes(["hardcore", "ultimate", "ironman", "craftsman", "hunted"])
three = format_gamemodes(["hardcore", "hunted"])
four = parse_gamemode(two)
five = parse_gamemode(three)

print(f'{one=}')
print(f'{two=}')
print(f'{three=}')
print(f'{four=}')
print(f'{five=}')

In [33]:
mojang = API()
mojang.get_uuid("umineko0o0_")

'5651a4116471402cb5999042c4de5088'

In [2]:
from typing import TypedDict

Test = TypedDict("Test", {"a": int, "b": str})
for a in Test.__annotations__:
    print(a)

a
b


In [2]:
a = [[1, 2], [3, 4], [5, 6]]
b = [l for l in a]
print(b)

[[1, 2], [3, 4], [5, 6]]
